In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta

In [2]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [62]:

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
title = "DH_studies_map"
my_nodesdict = {}
my_nodes= []
my_edges = []


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}

for artist in db.minedArtists.find({"name":"Foo Fighters"}) :
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    dateini = 0
    for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= venue["name"]+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            node = {
                "id" : idd,
                "name" : venue["name"],
                "lat" : float(venue["latitude"]),
                "lng" : float(venue["longitude"]),
                "weight" : float(1),
                "start" : date.get("datetime").isoformat(),
                "end" : (date.get("datetime")+timedelta(days=1)).isoformat()
                }
        else:
            
            node = {
                "id" : idd,
                "name" : venue["name"],
                "lat" : float(venue["latitude"]),
                "lng" : float(venue["longitude"]),
                "weight" : float(my_nodesdict[idd]["data"].get("weight"))+1 ,
                "start" : my_nodesdict[idd]["data"].get("start"),
                "end" : date.get("datetime").isoformat()
                }
        if dateini == 0:
            dateini =1
        else:
            #print my_nodes[-1]["data"]["id"]
            edge = {
                "source" : my_nodes[-1]["data"]["id"],
                "target" : node["id"],
                "color" : "red"
#                "weight" : float(e["weight"])
            }
            #print edge
            my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges

        #print "coords",latitude,"/",longitude

Foo Fighters


In [59]:
print len(my_nodesdict)

643


In [51]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)
my_nodesdictList = []
for key, value in my_nodesdict.iteritems():
    
    my_nodesdictList.append(value)



ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'smfTNnOoupf0fkEVkh2cUP7ZQ5174r4a3Vr4_iaJQqC', u'userId': u'DaofbRenqYpqqNmtz'}}


In [52]:
print my_nodesdictList

[{'data': {'end': '2006-06-13T19:00:00', 'name': u'Regent Theatre', 'weight': 1.0, 'start': '2006-06-12T19:00:00', 'lat': 52.057152, 'lng': 1.160702, 'id': u'Regent Theatre521'}}, {'data': {'end': '2001-03-16T19:00:00', 'name': u'House Of Blues', 'weight': 1.0, 'start': '2001-03-15T19:00:00', 'lat': 28.367735, 'lng': -81.519822, 'id': u'House Of Blues28-81'}}, {'data': {'end': '2015-10-04T19:00:00', 'name': u'Centennial Olympic Park', 'weight': 2.0, 'start': '2015-10-04T19:00:00', 'lat': 33.7603812, 'lng': -84.393292, 'id': u'Centennial Olympic Park33-84'}}, {'data': {'end': '2011-06-23T18:30:00', 'name': u'Stockholms Stadion', 'weight': 1.0, 'start': '2011-06-22T18:30:00', 'lat': 59.3333333, 'lng': 18.05, 'id': u'Stockholms Stadion5918'}}, {'data': {'end': '2007-07-06T19:00:00', 'name': u'Dingwalls', 'weight': 1.0, 'start': '2007-07-05T19:00:00', 'lat': 51.541508, 'lng': -0.145892, 'id': u'Dingwalls510'}}, {'data': {'end': '2000-05-02T19:00:00', 'name': u'Barton Coliseum', 'weight': 1

In [61]:
title = "Foo Fighters2"
create_topogram(title, my_nodesdictList, my_edges)

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


Creating topogram 'Foo Fighters2'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Foo Fighters2', u'userId': u'DaofbRenqYpqqNmtz', u'_id': u'dJHizpmPCPpeHTred', u'slug': u'foo-fighters2', u'createdAt': u'2017-10-12T12:50:51.139Z'}, u'statusCode': 200}
topogram ID : dJHizpmPCPpeHTred
{u'status': u'success', 'status_code': 200, u'data': [u'f45ovdgr2zHNNcRFb', u'byuhsGPD75CfPA42S', u'99MEgWL2kkce6t8DM', u'gReZh5cftFn5E7R8r', u'km4eZLTMg6rXBSBgc', u'ZuKiNfmYGcmqCoQ93', u'm5gqZ4f4jkyRpPh5s', u'gSXxLykyDdyFh62Lt', u'JGC4YYKS6hnkhy2Pf', u'bjj2SuNyZCH7Nc5oD', u'C9v38hrmgznWs9Kst', u'tWRmf3QzaZBGuXupE', u'rCQ7Anxzyer3hoESY', u'567nhvB9ahzcGE6hk', u'vmMsQLwJ2nw2KvKv3', u'caCciLXYxjC9LemPj', u'3dC9m85sAXjwMJsn9', u'g3jXW6nWKgDWmLSqb', u'Za4mRn9C2q6DEjJNi', u'6RwrwhLbnHxt5TXzb', u'iwNTcbg2zbDv9gXcC', u'rD6BeN6QXBPZorwsu', u'm7AZLQrgKYdxYDdY4', u'k3iZHbA5REBhKhWrH', u'tWggA6h5SfjtACiZ7', u'wDRzZZzsRgJ625dZR', u'GLured2tKmk8Q6E8S', 

In [ ]:
    #print artist
    

        dates_coords.append( [latitude,longitude])
    #print "datec",date_coords
    datapushed={}
    datapushed["dates_coords_lat_long"]=dates_coords
    datapushed["id"]=artist["_id"]
    datapushed["slug"]=artist["slug"]
    print datapushed
    dict_coords[artist["slug"]]=datapushed
print dict_coords



with open('data/test_data_nodes.csv') as f :
    reader = DictReader(f)
    for n in reader :
        node = {
            "id" : n["id"],
            "name" : n["name"],
            "lat" : float(n["lat"]),
            "lng" : float(n["lng"]),
            "weight" : float(n["weight"]),
            "start" : n["year_start"],
            "end" : n["year_stop"]
            }
        my_nodes.append({ "data" : node })

print my_nodes

with open('data/test_data_edges.csv') as f :
    reader = DictReader(f)
    for e in reader :
        edge = {
            "source" : e["source"],
            "target" : e["target"],
            "weight" : float(e["weight"])
        }
        my_edges.append({ "data" : edge })

# print my_edges

def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)

    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)

# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)

create_topogram("Geo-time network", my_nodes, my_edges)